In [ ]:
import pandas as pd
!pip install openpyxl

# 파일 읽기
file_path = './data/한국어-영어 번역(병렬) 말뭉치/2_conversation_200226.xlsx'
df = pd.read_excel(file_path)

# 데이터 정보 확인
print(df.info())  # 데이터 구조 및 열 정보 확인
print(df.head())  # 첫 몇 행 출력

# 데이터 처리
df = df.dropna()  # 결측치 제거
df = df[['원문', '번역문']]  # 필요한 열만 선택


In [ ]:
df

In [ ]:
kor = df['원문']
eng = df['번역문']

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
passive_rules = [
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBN'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'RB'}, {'TAG': 'VBN'}],
]

matcher.add("Passive", passive_rules)

# 수동문 개수 계산 함수
def count_passive_sentences(text):
    doc = nlp(text)  # 텍스트 처리
    matches = matcher(doc)  # Matcher로 패턴 매칭
    return len(matches)  # 매칭된 패턴 개수 반환

eng_passive_counts = []
for text in tqdm(eng, desc="Processing Sentences"):
    count = count_passive_sentences(text)
    eng_passive_counts.append(count)


# 결과를 데이터프레임에 추가
df["eng_passive_count"] = eng_passive_counts

# passive_count가 1 이상인 행만 남기기
df = df[df["eng_passive_count"] > 0].reset_index(drop=True)


In [ ]:
df

In [ ]:
kor = df['원문']
eng = df['번역문']

In [ ]:
#%pip install -U konlpy
import re
from konlpy.tag import Kkma
kkma = Kkma()

# 피동문 유형별 패턴 감지 함수
def count_korean_passive_sentences(text):
    # 형태소 분석
    tokens = kkma.pos(text)
    passive_count = 0
    # 피동 접미사 "-이/히/리/기-" 패턴
    passive_suffix = any(tok[1] == 'VV' and tok[0].endswith(('이', '히', '리', '기')) for tok in tokens)
    
    # 보조동사 "-어지-" 패턴
    passive_auxiliary = any(
        tokens[i][1] == 'VV' and tokens[i+1][0] == '어' and tokens[i+2][0] == '지' and tokens[i+2][1] == 'VXV'
        for i in range(len(tokens) - 2)
    )
    
    # "-되다", "-받다", "-당하다" 패턴
    passive_verb = any(tok[1] == 'VV' or tok[1] == 'XSV' and re.search(r'(되|받|당하)', tok[0]) for tok in tokens)
   
    #피동문 카운트
    if passive_suffix:
        passive_count += 1
    if passive_auxiliary:
        passive_count += 1
    if passive_verb:
        passive_count += 1
    
    return passive_count

In [62]:
count_korean_passive_sentences("교육 안내 메일 하단에 작은 글씨로 적혀있어요.")

2

In [ ]:
kor_passive_counts = []
for text in tqdm(kor, desc="Processing Sentences"):
    count = count_korean_passive_sentences(text)
    kor_passive_counts.append(count)


# 결과를 데이터프레임에 추가
df["kor_passive_count"] = kor_passive_counts

In [59]:
pd.set_option('display.max_columns', None)

In [60]:
df

,원문,번역문,eng_passive_count,kor_passive_count
0,우리 제품은 어떤 방식으로 도쿄에 보내지나요?,How will our products be shipped to Tokyo?,1,1
1,먼저 배로 도쿄 근처 항구까지 운반하고 그 후 차를 이용합니다.,"They'll be sent to Tokyo port by the ship, the...",1,0
2,교육 안내 메일 하단에 작은 글씨로 적혀있어요.,It was written in very small letters at the bo...,1,2
3,7월 29일부터 8월 2일까지가 공식적인 기간이에요.,It's officially scheduled from July 29 to Augu...,1,0
4,"네, 메일로 이미 공지를 했으니 다들 알고 있을 거에요.","Sure, but they'd already known since it was no...",1,1
5,인턴 5명을 포함해서 많은 직원을 뽑았다고 들었어요.,I was told that you hired many employees inclu...,1,1
6,그럼 광고 외에 출시까지 남은 과정은 무엇이 있나요?,"Then, what procedures are left before the rele...",1,1
7,"네, 저희 층의 직원들은 모두 제출했다고 들었어요.","Yes, I was told that every employee on my floo...",1,1
8,그보다 훨씬 더 많이 모여서 모두 뿌듯해했어요.,"It was raised way more than we planned, so we ...",1,2
9,좋은 아이디어네요. 먼저 전체 회의 때 통과된다면요.,That's a great idea. I hope it'll be approved ...,1,1


In [ ]:
h

In [ ]:
print('nsubjpass:', spacy.explain("nsubjpass"))
print('aux:', spacy.explain("aux"))
print('VBN:', spacy.explain("VBN"))
print('VBZ:', spacy.explain("VBZ"))
print('RB:', spacy.explain("RB"))